In [2]:
import numpy as np
#%matplotlib inline
#import matplotlib.pyplot as plt
from PIL import Image  , ImageDraw 
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
import json

In [0]:
torch.__version__

'1.3.1'

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

drive  sample_data


In [3]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}

In [4]:

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])
        self.transform = hm_transform
        #print(path)
        #print(type(path))
        #all_images = []
        #all_labels = []
        #all_hm_face = []
        #all_hm_hand = []
        # target_acquired - 0: image path -- 1: label
        #for i in len(path):

        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0])
        hm_face , hm_hand = self.get_heatmap(ann_path + '.txt',img_w,img_h)  
        hm_pose = self.get_heatmap_pose(ann_path + '.json',img_w,img_h)
        #image = np.dstack((image, hm_face))

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)
            hm_hand = hm_transform(hm_hand)
            hm_pose = torch.stack([ hm_transform(x) for x in hm_pose ], dim=1).squeeze(0)


        #print( hm_pose.size() )
        image = torch.cat((image,hm_face,hm_hand,hm_pose))
       
        #print(image.size())
        #if self.target_transform is not None:
        #    target = self.target_transform(target)
        
        
        # For Face
            #image = np.dstack((image, hm_face))

            #all_images.append(img_path)
            #all_labels.append(label)
            #all_hm_face.append(hm_face)
            #all_hm_hand.append(hm_hand)

        #return path, all_hm_face, all_hm_hand, target
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1

    def get_heatmap_pose(self,ann_path, img_w, img_h):
        
        heatmaps = [ Image.new('L', (img_w, img_h)) for x in range(17) ]

        try:
            with open(ann_path,'r') as f:
                data = json.load(f)
            #print(ann_path)
            #print(data)  
            try:
                keypoints = data[0]['keypoints']
            except IndexError:
                return heatmaps

            #print(keypoints)
            # TODO check for two key mappings -- working

            for i in range(0,len(keypoints),3):

                xmin = keypoints[i] - 5
                ymin = keypoints[i+1] - 5
                xmax = keypoints[i] + 5
                ymax = keypoints[i+1] + 5

                hmDraw = ImageDraw.Draw(heatmaps[i//3]) 
                hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")

        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return heatmaps



"""
Test Code


path, target = self.samples[index]
sample = self.loader(path)
if self.transform is not None:
  sample = self.transform(sample)
if self.target_transform is not None:
  target = self.target_transform(target)
# this is where you load your resnet data
resnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth
resnet = torch.load(resnet_path)  # load the stored features
return sample, resnet, target

#dataset = MyImageFolder("falling_dataset/val/")
#dataset[0]
#h = dataset.get_heatmap_pose("falling_dataset/train/Falling/11107.json",1280,720)
#print(len(h))
#for i in range(len(h)):
#    h[i].save("Annotated/train/%d.jpg"%(i))

path = "./falling_dataset/train/Falling"
!pwd
for file in os.listdir(path):
    if file.endswith(".json"):
        #print(os.path.join(path, file))

        with open(os.path.join(path, file)) as f:
            string = f.readline()
            if string == '[]':
                print(os.path.join(path, file))



#re, labels = dataset[1:3]
#print(re)
#print(labels)
#re[2].save('pred_face.jpeg')
#re[3].save('pred_hand.jpeg')
"""

'\nTest Code\n\n\npath, target = self.samples[index]\nsample = self.loader(path)\nif self.transform is not None:\n  sample = self.transform(sample)\nif self.target_transform is not None:\n  target = self.target_transform(target)\n# this is where you load your resnet data\nresnet_path = os.path.join(os.path.splitext(path)[0], \'.pth\')  # replace image extension with .pth\nresnet = torch.load(resnet_path)  # load the stored features\nreturn sample, resnet, target\n\n#dataset = MyImageFolder("falling_dataset/val/")\n#dataset[0]\n#h = dataset.get_heatmap_pose("falling_dataset/train/Falling/11107.json",1280,720)\n#print(len(h))\n#for i in range(len(h)):\n#    h[i].save("Annotated/train/%d.jpg"%(i))\n\npath = "./falling_dataset/train/Falling"\n!pwd\nfor file in os.listdir(path):\n    if file.endswith(".json"):\n        #print(os.path.join(path, file))\n\n        with open(os.path.join(path, file)) as f:\n            string = f.readline()\n            if string == \'[]\':\n                pr

In [6]:
input_path = './alpha-dataset'

image_datasets = {
    'train': 
    MyImageFolder(input_path + '/train_m', data_transforms['train']),
    'validation': 
    MyImageFolder(input_path + '/val_m', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [8]:
model = models.resnet50(pretrained=True).to(device)

model.cuda()
#print(model)

depth = 22    # ( 3 + 1 + 1 + 17 )


for param in model.parameters():
    param.requires_grad = False   


#------------ Modify First Layer
model.conv1 = nn.Conv2d(depth, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
model.conv1.requires_grad = True

#------------ Modify Last Layer
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2),
               nn.Sigmoid()).to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

In [12]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        
        f = open('loss/alpha/raw-face-hand-pose.txt','a+')
        
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        f.write('Epoch {}/{}\n'.format(epoch+1, num_epochs))
        

        for phase in ['train','validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            
            with torch.set_grad_enabled(phase == 'train'):
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outputs = model(inputs)
                  loss = criterion(outputs, labels)

                  if phase == 'train':
                      
                      optimizer.zero_grad()
                      loss.backward()
                      optimizer.step()

                  _, preds = torch.max(outputs, 1)
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

                  count = count+1                
                  if (count%40==0):
                    print (count)
             

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            f.write('{} loss: {:.4f}, acc: {:.4f}\n'.format(phase,epoch_loss,epoch_acc))
              
            
        torch.save(model, 'weights/alpha/raw-face-hand-pose/epoch_%d.pth'%(epoch))
        f.close()


    return model

Epoch 1/100
----------
40
train loss: 0.5718, acc: 0.7435
validation loss: 0.5776, acc: 0.7193
Epoch 2/100
----------
40
train loss: 0.5294, acc: 0.7832
validation loss: 0.5266, acc: 0.7789
Epoch 3/100
----------
40
train loss: 0.5112, acc: 0.8012
validation loss: 0.5239, acc: 0.7897
Epoch 4/100
----------
40
train loss: 0.4953, acc: 0.8192
validation loss: 0.4985, acc: 0.8014
Epoch 5/100
----------
40
train loss: 0.4829, acc: 0.8329
validation loss: 0.5331, acc: 0.7681
Epoch 6/100
----------
40
train loss: 0.4832, acc: 0.8254
validation loss: 0.5245, acc: 0.7807
Epoch 7/100
----------
40
train loss: 0.4832, acc: 0.8231
validation loss: 0.4822, acc: 0.8204
Epoch 8/100
----------
40
train loss: 0.4693, acc: 0.8442
validation loss: 0.4894, acc: 0.8168
Epoch 9/100
----------
40
train loss: 0.4616, acc: 0.8504
validation loss: 0.4726, acc: 0.8339
Epoch 10/100
----------
40
train loss: 0.4646, acc: 0.8481
validation loss: 0.4703, acc: 0.8384
Epoch 11/100
----------
40
train loss: 0.4594, ac

validation loss: 0.4213, acc: 0.8890
Epoch 87/100
----------
40
train loss: 0.3894, acc: 0.9240
validation loss: 0.4247, acc: 0.8854
Epoch 88/100
----------
40
train loss: 0.3823, acc: 0.9292
validation loss: 0.4253, acc: 0.8818
Epoch 89/100
----------
40
train loss: 0.3846, acc: 0.9269
validation loss: 0.4605, acc: 0.8412
Epoch 90/100
----------
40
train loss: 0.4120, acc: 0.8957
validation loss: 0.4183, acc: 0.8917
Epoch 91/100
----------
40
train loss: 0.3869, acc: 0.9256
validation loss: 0.4682, acc: 0.8366
Epoch 92/100
----------
40
train loss: 0.3928, acc: 0.9186
validation loss: 0.4209, acc: 0.8908
Epoch 93/100
----------
40
train loss: 0.3816, acc: 0.9315
validation loss: 0.4188, acc: 0.8908
Epoch 94/100
----------
40
train loss: 0.3877, acc: 0.9240
validation loss: 0.4434, acc: 0.8637
Epoch 95/100
----------
40
train loss: 0.3882, acc: 0.9238
validation loss: 0.4197, acc: 0.8908
Epoch 96/100
----------
40
train loss: 0.3797, acc: 0.9310
validation loss: 0.4197, acc: 0.8917
Epo

In [0]:
torch.save(model_ trained.state_dict(), 'weights/pytorch/weightsE3.h5')

In [0]:
# model = models.resnet50(pretrained=False).to(device)
# model.fc = nn.Sequential(
#                nn.Linear(2048, 128),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 2)).to(device)
epoch = 99
model = (torch.load('weights/raw-face-hand-pose-d/rfhp_epoch_%d.pth'%epoch))

In [0]:
!ls

darknet		      examples	      opt.py		train_sppe
data		      fallmodel.pth   output		video_ann.py
datalabeler_101.py    fi_data	      PoseFlow		video_demo.py
datalabeler_111.py    fn.py	      pPose_nms.py	webcam_demo.py
datalabeler_249.py    img.png	      __pycache__	weights
dataloader.py	      LICENSE	      README.md		yolo
dataloader_webcam.py  matching.py     requirements.txt
demo.py		      models	      resnetWeights
doc		      online_demo.py  SPPE


In [0]:
import glob
import random
pathFalling = '/content/drive/My Drive/AlphaPose/AlphaPose-pytorch/fi_data/train/Falling/'
pathNonFalling = '/content/drive/My Drive/AlphaPose/AlphaPose-pytorch/fi_data/train/NonFalling/'
accurateFalling = 0
accurateNonFalling = 0

list1= [f for f in glob.glob(pathFalling + "*.jpg", recursive=True)]
list2= [f for f in glob.glob(pathNonFalling + "*.jpg", recursive=True)]
validation_img_pathsFalling=random.sample(list1,100)
validation_img_pathsNonFalling = random.sample(list2,100)
img_listFalling = [Image.open('' + img_path) for img_path in validation_img_pathsFalling]
img_listNonFalling = [Image.open('' + img_path) for img_path in validation_img_pathsNonFalling]
print (len(img_listFalling))
print (len(img_listNonFalling))

100
100


In [0]:
validation_batchFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listFalling])
validation_batchNonFalling = torch.stack([data_transforms['validation'](img).to(device)
                                for img in img_listNonFalling])

In [0]:
pred_logits_tensorFalling = model(validation_batchFalling)
pred_logits_tensorFalling
pred_logits_tensorNonFalling = model(validation_batchNonFalling)
pred_logits_tensorNonFalling

RuntimeError: ignored

In [0]:
pred_probsFalling = F.softmax(pred_logits_tensorFalling, dim=1).cpu().data.numpy()
print (pred_probsFalling)
pred_probsNonFalling = F.softmax(pred_logits_tensorNonFalling, dim=1).cpu().data.numpy()
print (pred_probsNonFalling)

[[0.44844374 0.5515562 ]
 [0.12725133 0.8727487 ]
 [0.09804448 0.9019555 ]
 [0.00425655 0.99574345]
 [0.8194356  0.18056442]
 [0.24643923 0.7535607 ]
 [0.13835269 0.8616473 ]
 [0.1574074  0.84259254]
 [0.8485861  0.1514139 ]
 [0.32346088 0.6765391 ]
 [0.41357878 0.58642125]
 [0.15117285 0.8488271 ]
 [0.6003375  0.39966255]
 [0.02290936 0.9770906 ]
 [0.10023551 0.89976454]
 [0.29872367 0.70127636]
 [0.04371396 0.956286  ]
 [0.61934406 0.38065597]
 [0.30597326 0.6940267 ]
 [0.16753031 0.8324697 ]
 [0.66433    0.33567002]
 [0.84191    0.15808995]
 [0.6628018  0.33719823]
 [0.4977156  0.50228435]
 [0.87176895 0.12823105]
 [0.02149463 0.9785054 ]
 [0.48655316 0.5134468 ]
 [0.44041517 0.55958486]
 [0.1114507  0.88854927]
 [0.5616096  0.4383904 ]
 [0.570884   0.42911607]
 [0.32838956 0.6716105 ]
 [0.42478982 0.5752102 ]
 [0.5371839  0.46281612]
 [0.02694354 0.9730565 ]
 [0.61242115 0.3875789 ]
 [0.21000993 0.7899901 ]
 [0.32981074 0.6701893 ]
 [0.23768397 0.76231605]
 [0.846486   0.15351407]


In [0]:
i=0
accurateFalling=0
for i in range(len(pred_probsFalling)):
  if (pred_probsFalling[i][0]> 0.5):
    accurateFalling= accurateFalling +1 
i=0
accurateNonFalling=0
for i in range(len(pred_probsNonFalling)):
  if (pred_probsNonFalling[i][1]> 0.5):
    accurateNonFalling= accurateNonFalling +1 


In [0]:
overallAccuracy = (accurateNonFalling+accurateFalling)/(len(img_listFalling)+len(img_listNonFalling))

In [0]:
print (accurateFalling)
print (accurateNonFalling)

41
92


In [0]:
print (overallAccuracy)

0.665


In [0]:
ig, axs = plt.subplots(1, len(img_listFalling), figsize=(150, 150))
for i, img in enumerate(img_listFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsFalling[i,0],
                                                          100*pred_probsFalling[i,1]))
    ax.imshow(img)

In [0]:
ig, axs = plt.subplots(1, len(img_listNonFalling), figsize=(150, 150))
for i, img in enumerate(img_listNonFalling):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}%Falling , {:.0f}% Non-Falling".format(100*pred_probsNonFalling[i,0],
                                                          100*pred_probsNonFalling[i,1]))
    ax.imshow(img)

In [0]:
epoch = 99
device = 'cuda'
model = torch.load('weights/raw-face-hand-pose-d/rfhp_epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()

ResNet(
  (conv1): Conv2d(22, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [0]:
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['validation']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 60
total 100
0.6
